In [1]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# !pip install --upgrade xlrd

In [5]:
!ls "/content/drive/My Drive"

'Colab Notebooks'    HW_03.ipynb	  web_2.ipynb	  'отзывы за лето.xls'
 data.csv	     HW_4.ipynb		  web_3.ipynb
 dl-nlp-cnn1.ipynb   Lecture03.ipynb	  web_4.ipynb
 HW_02.ipynb	    'Lecture rnn.ipynb'   webinar4.ipynb


In [6]:
data = pd.read_excel("/content/drive/My Drive/отзывы за лето.xls")
data = data.dropna()
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

data['Content'] = data['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
import numpy as np
import keras
import tensorflow
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, SpatialDropout1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [10]:
text_corpus_data = data['Content'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_data = tokenizer.texts_to_sequences(text_corpus_data)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X = pad_sequences(sequences_data, maxlen=training_length)



In [12]:
Y = pd.get_dummies(data['Rating']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (20656, 5)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(18590, 132) (18590, 5)
(2066, 132) (2066, 5)


In [14]:
max_words = 20000
MAX_SEQUENCE_LENGTH = 50
output_dim = 264
epochs = 5
batch_size = 64
early_stopping=EarlyStopping(monitor='val_loss')  

**LSTM**

In [15]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=output_dim, input_length=X.shape[1]))
# model.add(Masking(mask_value=0.0))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/5
262/262 [==============================] - 174s 650ms/step - loss: 0.7394 - accuracy: 0.7465 - val_loss: 0.6354 - val_accuracy: 0.7805
Epoch 2/5
262/262 [==============================] - 168s 640ms/step - loss: 0.5781 - accuracy: 0.7968 - val_loss: 0.6290 - val_accuracy: 0.7789


In [16]:
score = model.evaluate(X_test, Y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 2s 462ms/step - loss: 0.6688 - accuracy: 0.7740


Test score: 0.6687614917755127
Test accuracy: 0.7739593386650085


***GRU***

In [17]:
model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=output_dim, input_length=X.shape[1]))
# model.add(Masking(mask_value=0.0))
model.add(SpatialDropout1D(0.2))
model.add(GRU(100, recurrent_dropout=0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
262/262 [==============================] - 137s 510ms/step - loss: 0.7486 - accuracy: 0.7412 - val_loss: 0.6560 - val_accuracy: 0.7800
Epoch 2/5
262/262 [==============================] - 133s 507ms/step - loss: 0.5686 - accuracy: 0.8007 - val_loss: 0.6421 - val_accuracy: 0.7773


In [18]:
score = model.evaluate(X_test, Y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 2s 318ms/step - loss: 0.6696 - accuracy: 0.7677


Test score: 0.6696377992630005
Test accuracy: 0.7676669955253601


**CNN**

In [19]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=output_dim, input_length=X.shape[1]))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
262/262 [==============================] - 51s 194ms/step - loss: 0.7723 - accuracy: 0.7488 - val_loss: 0.6405 - val_accuracy: 0.7778
Epoch 2/5
262/262 [==============================] - 52s 199ms/step - loss: 0.5805 - accuracy: 0.8013 - val_loss: 0.6257 - val_accuracy: 0.7805


In [21]:
score = model.evaluate(X_test, Y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 1s 184ms/step - loss: 0.6555 - accuracy: 0.7720


Test score: 0.6555097699165344
Test accuracy: 0.7720232605934143


***CNN+LSTM***

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=output_dim, input_length=X.shape[1]))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
262/262 [==============================] - 93s 336ms/step - loss: 0.7882 - accuracy: 0.7454 - val_loss: 0.6492 - val_accuracy: 0.7703
Epoch 2/5
262/262 [==============================] - 78s 297ms/step - loss: 0.6215 - accuracy: 0.7847 - val_loss: 0.6632 - val_accuracy: 0.7725


In [23]:
score = model.evaluate(X_test, Y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 1s 196ms/step - loss: 0.6859 - accuracy: 0.7652


Test score: 0.6858580112457275
Test accuracy: 0.7652468681335449
